In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import youtubecollector

In [12]:
from youtubecollector import channels

In [7]:
recommendations.get_recommendations()

<module 'youtubecollector.channels' from '/Users/omeuwese/Documents/Olaf/programming/python/correspondent/YouTubeExtremism/DataCollection/src/youtubecollector/channels.py'>